In [156]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import json

In [147]:
census_data = pd.read_csv('./data/FMCSA_CENSUS1_2020Aug.txt', encoding = "ISO-8859-1")

In [148]:
dot_numbers = census_data.DOT_NUMBER.to_list()

In [149]:
len(dot_numbers)

1717663

In [153]:
import json

In [160]:
def get_cargo(soup):
    data = {}
    cargo = soup.find("ul", attrs={"class": "cargo"}).find_all('li')
    for i in cargo:
        active = bool(i.get('class'))
        val = i.text
        if active:
            val = val.lstrip('X').strip()
        data[val] = active
    return data

In [161]:
def get_vehicle_type(soup):
    data = {}
    cargo = soup.find('table')
    headers = []
    for th in cargo.thead.find_all('th'):
        headers.append(th.text.strip())
    for row in cargo.tbody.find_all('tr'):
        row_data = {}
        row_items = row.text.strip().split('\n')
        for item, header in zip(row_items, headers):
            row_data[header] = item.strip()
        data[row_items[0]]= row_data
    return data

In [151]:
def get_carrier_registration(carrier_id):
    page = requests.get('https://ai.fmcsa.dot.gov/SMS/Carrier/{}/CarrierRegistration.aspx'.format(carrier_id)).text
    soup = BeautifulSoup(page)
    return {carrier_id: {'cargo': get_cargo(soup), 'types': get_vehicle_type(soup)}}

In [145]:
get_cargo(soup)

{'GENERAL FREIGHT': True,
 'HOUSEHOLD GOODS': False,
 'METAL; SHEETS, COILS, ROLLS': False,
 'MOTOR VEHICLES': False,
 'DRIVE AWAY/TOWAWAY': False,
 'LOGS, POLES, BEAMS, LUMBER': False,
 'BUILDING MATERIALS': False,
 'MOBILE HOMES': False,
 'MACHINERY, LARGE OBJECTS': False,
 'FRESH PRODUCE': False,
 'LIQUIDS/GASES': False,
 'INTERMODAL CONTAINERS': False,
 'PASSENGERS': False,
 'OIL FIELD EQUIPMENT': False,
 'LIVESTOCK': False,
 'GRAIN, FEED, HAY': False,
 'COAL, COKE': False,
 'MEAT': False,
 'GARBAGE, REFUSE, TRASH': False,
 'U.S. MAIL': False,
 'CHEMICALS': False,
 'COMMODITIES DRY BULK': False,
 'REFRIGERATED FOOD': False,
 'BEVERAGES': False,
 'PAPER PRODUCTS': False,
 'UTILITY': False,
 'FARM SUPPLIES': False,
 'CONSTRUCTION': False,
 'WATER WELL': False,
 'OTHER': False}

In [144]:
get_vehicle_type(soup)

{'Straight Trucks': {'Vehicle Type': 'Straight Trucks',
  'Owned': '98,785',
  'Term Leased': '0',
  'Trip Leased': '0'},
 'Truck Tractors': {'Vehicle Type': 'Truck Tractors',
  'Owned': '27,164',
  'Term Leased': '0',
  'Trip Leased': '0'},
 'Trailers*': {'Vehicle Type': 'Trailers*',
  'Owned': '96,285',
  'Term Leased': '0',
  'Trip Leased': '0'},
 'Hazmat Cargo Tank Trailers*': {'Vehicle Type': 'Hazmat Cargo Tank Trailers*',
  'Owned': '0',
  'Term Leased': '0',
  'Trip Leased': '0'},
 'Hazmat Cargo Tank Trucks': {'Vehicle Type': 'Hazmat Cargo Tank Trucks',
  'Owned': '0',
  'Term Leased': '0',
  'Trip Leased': '0'},
 'Motor Coach': {'Vehicle Type': 'Motor Coach',
  'Owned': '0',
  'Term Leased': '0',
  'Trip Leased': '0'},
 'School Bus 1-8*': {'Vehicle Type': 'School Bus 1-8*',
  'Owned': '0',
  'Term Leased': '0',
  'Trip Leased': '0'},
 'School Bus 9-15': {'Vehicle Type': 'School Bus 9-15',
  'Owned': '0',
  'Term Leased': '0',
  'Trip Leased': '0'},
 'School Bus 16+': {'Vehicle 

In [163]:
json.dumps(get_carrier_registration(21800))

'{"21800": {"cargo": {"GENERAL FREIGHT": true, "HOUSEHOLD GOODS": false, "METAL; SHEETS, COILS, ROLLS": false, "MOTOR VEHICLES": false, "DRIVE AWAY/TOWAWAY": false, "LOGS, POLES, BEAMS, LUMBER": false, "BUILDING MATERIALS": false, "MOBILE HOMES": false, "MACHINERY, LARGE OBJECTS": false, "FRESH PRODUCE": false, "LIQUIDS/GASES": false, "INTERMODAL CONTAINERS": false, "PASSENGERS": false, "OIL FIELD EQUIPMENT": false, "LIVESTOCK": false, "GRAIN, FEED, HAY": false, "COAL, COKE": false, "MEAT": false, "GARBAGE, REFUSE, TRASH": false, "U.S. MAIL": false, "CHEMICALS": false, "COMMODITIES DRY BULK": false, "REFRIGERATED FOOD": false, "BEVERAGES": false, "PAPER PRODUCTS": false, "UTILITY": false, "FARM SUPPLIES": false, "CONSTRUCTION": false, "WATER WELL": false, "OTHER": false}, "types": {"Straight Trucks": {"Vehicle Type": "Straight Trucks", "Owned": "98,785", "Term Leased": "0", "Trip Leased": "0"}, "Truck Tractors": {"Vehicle Type": "Truck Tractors", "Owned": "27,164", "Term Leased": "0", 